In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [10]:
bucket = 'musicgenredatalake'
data_key='standardized/datasets/features_3_sec.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
df = pd.read_csv(data_location)

In [11]:
def mahalanobis(x=None, data=None, cov=None):

    x_mu = x - np.mean(data)
    #if not cov:
        #cov = np.cov(x.T)
    inv_cov = np.linalg.inv(cov)
    mahalanobis_distances = np.diag(np.dot((x_mu @ inv_cov), x_mu.T))
   
    return mahalanobis_distances

In [12]:

df['label'] = df['label'].astype('category')
df['length'] = pd.to_numeric(df['length'])
df_x = df.drop(['filename', 'label', 'length'], axis=1)
x = np.array(df_x)
y = df['label'].cat.codes.values
scaler = MinMaxScaler()
x_norm = scaler.fit_transform(x)
data = df.iloc[:, 2:-1]
data_norm = scaler.fit_transform(data)
cov_h_norm = np.cov(x_norm.T)
cov_i_norm = cov_h_norm + 10 * np.eye(57, 57)
mahalanobis_dis_i_norm = mahalanobis(x=x_norm, data=data_norm, cov=cov_i_norm)
p25 = np.percentile(mahalanobis_dis_i_norm, 25)
p75 = np.percentile(mahalanobis_dis_i_norm, 75)
threshold = p75 + 3 * (p75 - p25)
x_fn = x_norm[mahalanobis_dis_i_norm < threshold]
y_fn = y[mahalanobis_dis_i_norm < threshold]
cols = df_x.columns
df_X_fn = pd.DataFrame(x_fn)

In [13]:
cols = ['chroma_stft_mean', 'chroma_stft_var', 'rms_mean',
           'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var',
           'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
           'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
           'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo',
           'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
           'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
           'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
           'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
           'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
           'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
           'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
           'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var', 'label']

In [14]:
df_X_fn['57']=y_fn
df_fn=df_X_fn
df_fn.columns=cols

In [15]:
df_fn.to_csv('s3://musicgenredatalake/trusted/datasets/features_3_sec.csv', index=False)